In [1]:
import pyarrow as pa

In [2]:
!echo $JAVA_HOME

/usr/java/jdk1.8.0_181-cloudera


In [3]:
fs = pa.hdfs.HadoopFileSystem(
    host='cnt7-naya-cdh63',
    port=8020,
    user='hdfs',
    kerb_ticket=None,
    driver='libhdfs',
    extra_conf=None)

In [61]:
cap = fs.get_capacity()

print(cap)
print(str(round(cap / pow(1024,2), 0)) + ' MB')

96414515200
91948.0 MB


In [62]:
###
# folder = 'hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging'
# try:
#     # Del if exist /tmp/sqoop/staging.
#     fs.rm(folder, recursive=True)
# except pa.ArrowIOError:
#     if not fs.isdir(folder):
#         raise


# Finally, using ls() we verify the content of the folder
sql_dir_files = fs.ls('/tmp/hive/hive', detail=False)
print(sql_dir_files)

['/tmp/hive/hive/19acffe3-acc4-43ab-ad74-b355ee5868a8', '/tmp/hive/hive/2578e566-5503-47ca-bb44-16b1d3df0270', '/tmp/hive/hive/338a963d-da54-4df1-93f7-482007d33f72', '/tmp/hive/hive/3aee8966-4e93-499e-8f04-17c7ca903763', '/tmp/hive/hive/6915a150-1168-4785-858a-36fec5fff258', '/tmp/hive/hive/6bfb7742-65d3-4b41-9b20-671f751ec554', '/tmp/hive/hive/76eda73c-22e3-4f8d-9285-c7dc5e7e80dc', '/tmp/hive/hive/a6062efd-ea92-487e-89b0-8c41f19f376a', '/tmp/hive/hive/c324e2de-e526-4a93-ae63-0014d83570a1', '/tmp/hive/hive/fbf7b2d6-179d-4f04-890c-6e81c8c9cd57']


In [19]:
### EXAMPLE2.py

'''
upload files created locally to the HDFS staging area.
download data from HDFS to our local system
'''
import os
import pyarrow as pa

fs = pa.hdfs.HadoopFileSystem(
    host='Cnt7-naya-cdh63',
    port=8020,
    user='hdfs',
    kerb_ticket=None,
    extra_conf=None)

#Note: will delete only if exist /tmp/sqoop/staging
if fs.exists('hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging'):
    fs.rm('hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging', recursive=True)


#First we use mkdir() to create a staging area in HDFS under /tmp/sqoop/staging.
fs.mkdir('hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging', create_parents=True)

#Next, we use upload() to copy all *.sql files created locally to the staging area.

local_path = '/home/naya/tutorial/sqoop/scripts/'
extension = '.sql'
sql_files = [f for f in os.listdir(local_path) if f.endswith(extension)]
print(sql_files)

#,'rb'
for f_name in sql_files:
    with open(local_path + f_name,'rb') as f:
        fs.upload('hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging/{}'.format(f_name), f)

# Finally, using ls() we verify the content of the folder
sql_dir_files = fs.ls('hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging', detail=False)
print(sql_dir_files)

#We can also download data from HDFS to our local system using the download() method.
print("Let's transfer files from hdfs to local OS:")
fs.download(path='hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging/classicmodels.sql',
            stream='/home/naya/tutorial/sqoop/classicmodels.sql',
            buffer_size=None)

# Now we can check if the file was transfer to a local directory
local_files = [f for f in os.listdir('/home/naya/tutorial/sqoop') if f.endswith(extension)]
for f_name in local_files:
    print(f_name)

['classicmodels.sql', 'ordsTable.sql']
['/tmp/sqoop/staging/classicmodels.sql', '/tmp/sqoop/staging/ordsTable.sql']
Let's transfer files from hdfs to local OS:
classicmodels.sql


In [20]:
### example3.py

'''
Managing permissions with chmod() & chown()
In this example we manage permissions for our files
'''

import os
import pyarrow as pa

fs = pa.hdfs.HadoopFileSystem(
    host='Cnt7-naya-cdh63',
    port=8020,
    user='hdfs',
    kerb_ticket=None,
    extra_conf=None)

# Change access permissions for the file classicmodels.sql in the staging directory.
fs.chmod('hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging/classicmodels.sql', 775)

# Change access permissions for a file in the staging directory in order
# to allow for naya user to get this file only
fs.chown('hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging/classicmodels.sql', owner='naya', group='naya')

# Finally, we can chaeck the details of the files either by the info() method.
a=fs.info('hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging/classicmodels.sql')
print(a)

# Alternatively we can do it using the detail=True argument of the ls() method.
file_details = fs.ls('hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging/classicmodels.sql', detail=True)
for detail in file_details:
    for k, v in detail.items():
        print(k, ':', v)

{'path': 'hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging/classicmodels.sql', 'owner': 'naya', 'group': 'naya', 'size': 202062, 'block_size': 134217728, 'last_modified': 1678656508, 'last_accessed': 1678656507, 'replication': 1, 'permissions': 775, 'kind': 'file'}
kind : file
name : /tmp/sqoop/staging/classicmodels.sql
owner : naya
group : naya
last_modified_time : 1678656508
last_access_time : 1678656507
size : 202062
replication : 1
block_size : 134217728
permissions : 775


# Exercise 1

Get the amount of used space on your file system.


# solution

In [68]:
fs = pa.hdfs.HadoopFileSystem(
    host='Cnt7-naya-cdh63',
    port=8020,
    user='hdfs',
    kerb_ticket=None,
    extra_conf=None)

used_space = fs.disk_usage('hdfs://Cnt7-naya-cdh63:8020/')
print(str(round(used_space / pow(1024,2), 0)) + ' MB')
print(used_space)

# 2. Get reported free space of the file system in bytes and modify this output into a human readable format.
path='hdfs://Cnt7-naya-cdh63:8020/'
cap=fs.get_capacity()
disk_usage=fs.disk_usage(path)
free_space=cap-disk_usage

print(str(round(free_space / pow(1024, 2), 0)) + ' MB')
print(free_space )

643.0 MB
674713158
91305.0 MB
95739802042


# Exercise 2


Use the delete method to delete some of the files from the staging directory.

# Solution

In [33]:
def create_files_example_2():
    #First we use mkdir() to create a staging area in HDFS under /tmp/sqoop/staging.
    fs.mkdir('hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging', create_parents=True)

    #Next, we use upload() to copy all *.sql files created locally to the staging area.

    local_path = '/home/naya/tutorial/sqoop/scripts/'
    extension = '.sql'
    sql_files = [f for f in os.listdir(local_path) if f.endswith(extension)]
    print(sql_files)

    #,'rb'
    for f_name in sql_files:
        with open(local_path + f_name,'rb') as f:
            fs.upload('hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging/{}'.format(f_name), f)

create_files_example_2()
# using ls() we verify the content of the folder
sql_dir_files = fs.ls('hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop', detail=False)
print(sql_dir_files)

fs.rm('hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging', recursive=True)


# Finally, using ls() we verify the content of the folder
sql_dir_files = fs.ls('hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop', detail=False)
print(sql_dir_files)


['classicmodels.sql', 'ordsTable.sql']
['/tmp/sqoop/staging']
[]


# Exercise 3

1. Connect to a local HDFS with parameters
2. Get reported free space of the file system in bytes and modify this output into a human readable format
3. Get disk usage information about /user directory in HDFS in bytes and modify this output into a human readable format
4. Get detailed information about a HDFS directory /user/hive/warehouse in human readable format
5. Create a directory in HDFS: first check if a directory exists and through the message about it, if a directory don't
 exist create it. Use a global scope's variable. Check the script with an existing path too.
6. Rename a file in HDFS and check if the file was renamed.
7. Delete the whole directory /tmp/sqoop/staging and it's files and check directory again.
8. Create a new file in any directory on your OS and write any row into it (hint: use input() method). Upload this file
to HDFS directory.



# solution

In [71]:
# 1. Connect to a local HDFS with parameters such: host, port, user, kerberos ticket for authentication if exists,
# driver and other configurations.
fs = pa.hdfs.HadoopFileSystem (
    host='Cnt7-naya-cdh63',
    port=8020,
    user='hdfs',
    kerb_ticket=None,
    extra_conf=None)

# 2. Get reported free space of the file system in bytes and modify this output into a human readable format.
used_space = pa.hdfs.HadoopFileSystem.disk_usage(fs,'hdfs://Cnt7-naya-cdh63:8020/')
cap=pa.hdfs.HadoopFileSystem.get_capacity(fs)
MB = 1024 ** 2
print(f"space {used_space//MB}MB / {cap//MB}MB")

# 3. Get disk usage information about /user directory in HDFS in bytes and modify this output
# into a human readable format.
used_space = pa.hdfs.HadoopFileSystem.disk_usage(fs,'/user')
print(f"used by hdfs:///user: {used_space//MB}MB")

# 4. Get detailed information about a HDFS directory /user/hive/warehouse in human readable format.
files = fs.ls('/user/hive/warehouse', detail=True)
for file in files:
    print(file['name'])
    for key,value in file.items():
        print('\t', key, ':', value)

# 5. Create a directory in HDFS: first check if a directory exists and through the message about it,
# if a directory don't exist create it.
# Use a global scope's variable. Check the script with an existing path too.
p = 'hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging/'
if fs.exists(p):
    print(p, 'already exists')
else:
    print('creating', p)
    fs.mkdir(p, create_parents=True)

# 6. Rename a file in HDFS and check if the file was renamed.
src = p+'stam.txt'
print('creating', src)
with fs.open(src, 'wb') as f:
    f.write('hello hdfs!\n'.encode())
    f.close()

print(fs.ls(p))
dst = p + 'hello.txt'
if fs.exists(dst):
    fs.rm(dst)
fs.rename(src, dst)

print(fs.cat(p+'hello.txt').decode())

# 7. Delete the whole directory /tmp/sqoop/staging and it's files and check directory again.
print('directory exists now:', fs.exists(p))
print('deleting', p)
fs.rm(p, recursive=True)
print('directory exists after deleting:', fs.exists(p))

# 8. Create a new file in any directory on your OS and write any row into it (hint: use input() method).
#    Upload this file to HDFS directory.
p = '/home/naya/tutorial/sqoop/scripts/file.txt'
with open(p, 'w') as f:
    print('we can upload!', file=f)
with open(p,'rb') as f:
    fs.upload('hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/development/file.txt', f)

print(fs.cat('hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/development/file.txt').decode())


space 643MB / 91948MB
used by hdfs:///user: 641MB
/user/hive/warehouse/classicmodels.db
	 kind : directory
	 name : /user/hive/warehouse/classicmodels.db
	 owner : hdfs
	 group : hive
	 last_modified_time : 1678656018
	 last_access_time : 0
	 size : 0
	 replication : 0
	 block_size : 0
	 permissions : 1023
creating hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging/
creating hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging/stam.txt
['/tmp/sqoop/staging/stam.txt']
hello hdfs!

directory exists now: True
deleting hdfs://Cnt7-naya-cdh63:8020/tmp/sqoop/staging/
directory exists after deleting: False
we can upload!

